In [1]:
#!/usr/bin/python
# -*- coding: UTF-8 -*-
from __future__ import division
import csv
import random
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
import time

In [2]:
sc = SparkContext()

- Transaction generate

In [3]:
#size: how many items in one transaction
#Length: number of transaction
#itemsize: number of items
def transaction_generation(size, length, itemsize):
    transaction = []
    item_range = 97 + itemsize
    for iterator in range(0, length):
        transaction.append(random.sample([chr(i) for i in range(97, item_range)], size))
    return transaction

- Data preparation

In [4]:
def readcsv(path):
    Tractions = []
    with open(path, 'r') as csvfile:
        Data = csv.reader(csvfile)
        for line in Data:
            Tractions.append(list(line))
    return Tractions

In [5]:
# def infrequentSubSet(candidates, frequentset):
#     size = len(frequentset[0]) + 1
#     freqSetNum = {}
#     for candidate in candidates:
#         freqSetNum[candidate] = 0
#         for freqset in frequentset:
#             freqset = set(freqset)          
#             if freqset.issubset(candidate):
#                 freqSetNum[candidate] += 1
#     freqSubSet = dict(filter(lambda x: x > size - 1 , freqSetNum.items()))
#     return freqSubSet.keys()
def infrequentSubSet(candidates, frequentset):
    size = len(frequentset[0]) + 1
    freqSetNum ={}
    for candidate in candidates:
        freqSetNum[candidate] = 0
        candidate_list = list(candidate)
        for i in range(size):
            sub_candidate = frozenset(candidate_list[:i]+candidate_list[(i+1):])
            if sub_candidate in list(frequentset):
                freqSetNum[candidate] += 1
    return [k  for (k,v) in freqSetNum.items() if v == size]
   

In [6]:
a = [[1,2],[1,4],[2,3],[2,4],[3,4]]
a = [frozenset(i) for i in a]
joinset(a)

NameError: name 'joinset' is not defined

In [7]:
def joinset(candidate):
#     print(4)
    length = len(candidate)
    size = len(candidate[0]) + 1
    output = []
    for i in range(length):
        for j in range(i, length):
            joinedSet = frozenset(list(candidate[i]) + list(candidate[j]))
            if len(joinedSet) == size:
                output.append(joinedSet)
#     print(5)
#     TractionSet_bc = sc.broadcast(TractionSet.collect())
    output = infrequentSubSet(output, candidate)
#     print(6)
    return output

- Main function

In [8]:
def aprioir(data, minsup):
    Size = len(data)
    Tractions = sc.parallelize(data)
    TractionSet = Tractions.map(lambda x: set(x)).cache()
    
    #find the 1-item pattern and frequency
    TractionOneSet = Tractions.flatMap(lambda x: frozenset(set(x))).cache()
    TractionSetValue = TractionOneSet.map(lambda x: (x, 1 / Size)).reduceByKey(lambda x,y: x + y)
    FrequentSetValue = TractionSetValue.filter(lambda x: x[1] >= minsup)
    FrequentOneSet = FrequentSetValue.map(lambda x: frozenset([x[0]])).collect()
    
    TractionSet_bc = sc.broadcast(TractionSet.collect())
#     TractionSet_bc = TractionSet.collect()


    k = 2
    output = {}
    output[k - 2] = FrequentOneSet
    while (len(output[k - 2]) > 0):
        if k-2 >= 6:
            break
        print '%s th interation'%k
#         print('before')
#         print(output[k-2])
        print(1)
        JoinedKSet = joinset(output[k-2])
#         print('after')
#         print(JoinedKSet)
        print(2)
        FrequentKSupport = sc.parallelize(JoinedKSet).map(lambda x: (x, len([trac for trac in TractionSet_bc.value if x.issubset(trac)]) / Size)).filter(lambda x: x[1] >= minsup).cache()
#         print(FrequentKSupport.take(5))
        FrequentKSet = FrequentKSupport.map(lambda x: x[0]).collect()
        print(3)
        k += 1
        output[k - 2] = FrequentKSet
#         break
    return output

In [11]:
%%time
if __name__ == "__main__":
    #size, length, itemsize
    #    Data = transaction_generation(5, 1000000, 10)
    #     Data = readcsv('75000i.csv')
    Data = readcsv('./transactions.txt')[:25000]
    minsup = 0.018

    start = time.time()
    freq_pattern = aprioir(Data, minsup)
    stop = time.time()

    for k, v in freq_pattern.items():
        print(k)
        print(len(v), v[:10])
    print 'The duration is', str(stop - start), 's'

2 th interation
1
2
3
3 th interation
1
2
3
4 th interation
1
2
3
0
(198, [frozenset(['21314']), frozenset(['21930']), frozenset(['22386']), frozenset(['22382']), frozenset(['22144']), frozenset(['21080']), frozenset(['23240']), frozenset(['22423']), frozenset(['84755']), frozenset(['23200'])])
1
(44, [frozenset(['21931', '22386']), frozenset(['22698', '22699']), frozenset(['82494L', '82482']), frozenset(['23202', '23203']), frozenset(['85099B', '22411']), frozenset(['22699', '22697']), frozenset(['22383', '20728']), frozenset(['22383', '20725']), frozenset(['22382', '20728']), frozenset(['23203', '85099B'])])
2
(1, [frozenset(['22698', '22699', '22697'])])
3
(0, [])
The duration is 40.0090339184 s
CPU times: user 1.89 s, sys: 211 ms, total: 2.1 s
Wall time: 40.2 s


In [14]:
data = readcsv('75000i.csv')
Size = len(data)
Tractions = sc.parallelize(data)
TractionSet = Tractions.map(lambda x: set(x)).cache()

#find the 1-item pattern and frequency
TractionOneSet = Tractions.flatMap(lambda x: frozenset(set(x))).cache() # all  elements unique
TractionSetValue = TractionOneSet.map(lambda x: (x, 1 / Size)).reduceByKey(lambda x,y: x + y)
FrequentSetValue = TractionSetValue.filter(lambda x: x[1] >= minsup)
FrequentOneSet = FrequentSetValue.map(lambda x: [x[0]]).collect()

TractionSet_bc = sc.broadcast(TractionSet.collect())

FrequentOneSet.map(lambda x:(x,1)).reduceByKey().collect


# k = 2
# output = {}
# output[k - 2] = FrequentOneSet
# while (len(output[k - 2]) > 0):
#     print '%s th interation'%k
#     print(output[k-2])
#     JoinedKSet = joinset(output[k-2])
#     print(JoinedKSet)
#     FrequentKSupport = sc.parallelize(JoinedKSet).map(lambda x: (x, len([trac for trac in TractionSet_bc.value if x.issubset(trac)]) / Size)).filter(lambda x: x[1] >= minsup).cache()

#     FrequentKSet = FrequentKSupport.map(lambda x: x[0]).collect()

#     k += 1
#     output[k - 2] = FrequentKSet
#     break

# # return output

AttributeError: 'list' object has no attribute 'map'

In [31]:
print(Tractions.flatMap(lambda x: frozenset(set(x))).take(5))
Tractions.map(lambda x: frozenset(set(x))).take(3)

['1', '21', '1', '11', '5']


[frozenset({'1', '21'}),
 frozenset({'1', '11', '5'}),
 frozenset({'1', '2', '7'})]